# System description
The system is made of 4 sub systems assembly:
- a CPU to generate thermal flow when running
- a fan to blow fresh air on the exhanger to cool it
- an exchangeur to dissipate the heat flow from CPU
- a controler to decide the power level of the fan from the CPU temperature

In [ ]:
# get system
from cpu.systems import CPUSystem
cpu = CPUSystem("cpu")

## design phase
Exchanger surface is evaluated so that maximum CPU temparature is 80°C when air temperature is 40° and CPU is running at its maximum.

In [ ]:
from cosapp.drivers import NonLinearSolver

design = cpu.add_driver(NonLinearSolver('solver'))
design.extend(cpu.design_methods["exchanger_surface"])

design.runner.set_values({"fan.T_air": 40., "T_cpu": 80., "cpu.usage": 100.})

# run design
cpu.run_drivers()

# simulation phase
CPU temperature is evaluated from its initial temperature, and CPU power level. 

In [ ]:
from cosapp.drivers import RungeKutta

time_driver = cpu.add_driver(RungeKutta(order=3, history = True))
solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))

time_driver.time_interval = (0, 30)
time_driver.dt = 0.5

# Define a simulation scenario
time_driver.set_scenario(
    init = {'T_cpu': 10.},
    values = {
        "fan.T_air": 40., 
        "cpu.usage": "100 if time <20 else 0."}
)

# identify data to store for interface viewer
from cosapp.recorders import DataFrameRecorder

rec = time_driver.add_recorder(DataFrameRecorder(includes=['T_cpu', 'cpu.power', 'cpu.usage', 'cpu.heat_flow', 'exchanger.heat_flow', 'exchanger.h', 
                                                'exchanger.surface', 'fan.tension', 'fan.fl_out.mass_flow', 'controler.T', 'controler.tension'], hold=False), period=0.5)

# run design + simulation
cpu.run_drivers()

In [ ]:
# start interface viewer
from cosapp_lab.widgets import SysExplorer

a = SysExplorer(cpu, template = "cpu_lab.json", anchor = "tab-after")